# Tutorial 2: Add MinIO to an existing VM (optional)

### **Requirements:** a VM running your MySQL database (tutorial 1 complete).


In this tutorial we'll cover how to launch MinIO from an existing VM (which was configured in the previous tutorial). 

MinIO is an object storage server useful for storing large files. If all of your data files are small (roughly <10 MB), you can probably just store all of your data directly in the MySQL database and skip this part. If you need to access larger files, then this tutorial is for you.

-----------------------

## 1. Mount your external volume

The specifics here will vary based on your setup. We'll cover some general steps, but if you're part of an organization, you should probably reach out to a system admin.

First, create a folder on your VM instance (we'll assume here that the folder is called `minio`) attach the volume to this folder on the VM instance (e.g. via your cloud computing platform, such as OpenStack, or by plugging in an external hard drive). Run `lsblk` or `df -h` to list all devices and check that it is attached (it will probably look something like `/dev/sdb1`). 

We'll make a new terminal session using tmux, a terminal multiplexer (which is a fancy way of saying that it opens terminal session which you can attach to or detatch from). We'll launching our MinIO service in this terminal, MinIO will keep running in this terminal even if you disconnect from the server or your network connection is interrupted. 

Let's create a new terminal session and name it "minio":

<pre style="background-color: #1E1E1E; color: white; padding: 10px; border-radius: 5px; border-left: 5px solid #007bff;">
tmux new -s minio
</pre>

Now you're in your new tmux terminal. 

(You can detach from the tmux session and return to your normal terminal by pressing `Ctrl+B D`. You can later reattach to the session using the command `tmux attach -t minio`.)

## 2. Launch the MinIO server

Now we'll launch our MinIO server container using Docker. The command below maps ports 9000 and 9001 from the host to the container for server and console access, mounts the host `minio` directory as a volume for data storage, and sets up root access credentials for the server. **Enter a password of your choosing.**

<pre style="background-color: #1E1E1E; color: white; padding: 10px; border-radius: 5px; border-left: 5px solid #007bff;">
sudo docker run -p 9000:9000 -p 9001:9001 --name minio -v ~/minio:/data -e "MINIO_ROOT_USER=root" -e "MINIO_ROOT_PASSWORD=&lt;password&gt;" quay.io/minio/minio:RELEASE.2021-10-23T03-28-24Z server /data -console-address ":9001"
</pre>

You should now be able to access your MinIO console via a web browser at `http://<your.ip>:9001`.

The command you've shared is actually two distinct parts:

Ctrl + b, Shift + %: This is a key combination used in the tmux terminal multiplexer. tmux allows you to create multiple terminal sessions within a single window, and it's controlled through various key bindings:

Ctrl + b is the default prefix in tmux that you press before using any tmux command.
Shift + % (after pressing Ctrl + b) splits the current pane vertically into two panes. This allows you to have two command-line interfaces side-by-side within the same tmux window.
sudo docker run --name minio1_client --net=host -it -v ~/minio:/data --entrypoint=/bin/sh minio/mc: This is a Docker command that runs a new container with specified parameters:

sudo: Runs the Docker command with superuser privileges, ensuring that it has the necessary permissions.
docker run: Creates and starts a new Docker container.
--name minio1_client: Names the new container minio1_client.
--net=host: Uses the host network for the container. This means the container shares the same network namespace as the host, effectively giving it full access to the host's network interfaces.
-it: Runs the container in interactive mode with a TTY, allowing you to interact with the container directly through the terminal.
-v ~/minio:/data: Mounts the ~/minio directory from the host system to the /data directory inside the container. This allows for data persistence and sharing between the host and the container.
--entrypoint=/bin/sh: Overrides the default entrypoint of the container with /bin/sh, starting a shell session inside the container. This is useful for debugging, configuration, or manual operation within the container environment.
minio/mc: Specifies the Docker image to use, in this case, minio/mc, which is the MinIO Client (mc). The MinIO Client provides a command-line interface to manage MinIO and Amazon S3 compatible services.
In summary, the tmux part of the command is for splitting your terminal window so you can run multiple sessions side-by-side. The docker run command starts a new container named minio1_client using the minio/mc image, attaches it to the host's network, opens an interactive shell session, and mounts a local directory (~/minio) to /data inside the container. This setup is likely intended for managing MinIO or S3-compatible services from the command line using MinIO Client within an isolated environment.


mc config host add minio http://127.0.0.1:9001 root "bu!k4MaLeinSc"